In [9]:
import momepy

In [10]:
import geopandas as gpd
import networkx as nx
from geopandas import GeoDataFrame
import os
import zipfile
import pandas as pd
import glob
from pandas import DataFrame
layer_crs= 'epsg:2039'
project_folder = os.path.dirname(os.getcwd())
data_folder = 'elements_as_graph'

In [8]:
# Start by merging files
# 1. Create new folder for each attribute
folder_names  = ['educational_institutes','synagogues','Leisure_amenities','Sport_facilities','Health_services','Playgrounds']
for name in folder_names:
    os.makedirs(project_folder + r'\TelAvivLayers\{}\{}'.format(data_folder,name),exist_ok = True)

In [9]:
# 2. uzip files
# folder_to_zip = glob.glob(project_folder + r'\TelAvivLayers\{}\*\*'.format(data_folder))
# for file_path in folder_to_zip:
#     with zipfile.ZipFile(file_path, 'r') as zip_ref:
#         zip_ref.extractall(os.path.dirname(file_path))

In [10]:
# 3. merge all files of the same kind
path_by_subgect = glob.glob(project_folder + r'\TelAvivLayers\{}\*'.format(data_folder))
for path in path_by_subgect:
    data_list = []
    name =(os.path.basename(path))
    print('run on {}'.format(name))
    files_path= glob.glob(r'{}\*.shp'.format(path))
    cols = ['geometry']
    for file_path in files_path:
        file =  gpd.read_file(file_path)[cols].reset_index()
        # Make sure the file is in the correct CRS and if not, transform it to the right one
        if file .crs.srs !=layer_crs:
            print('different crs')
            file.to_crs(layer_crs)
        file_name =  file_path.split('\\')[-1]
        print('in {} has {} {} '.format(file_name ,len(file),name))
        file['name'] = file_name
        data_list.append(file)
    # Combine all and leave only one among those  with the same geometry
    one_file = pd.concat(data_list)
    print('there is  {} {}'.format(len(one_file),name))
    one_file.to_file('output/streets_elements/elements/{}/{}.shp'.format(data_folder,name))

run on educational_institutes
in Kindergartens.shp has 662 educational_institutes 
in Schools.shp has 211 educational_institutes 
there is  873 educational_institutes
run on Health_services
in Family Health.shp has 15 Health_services 
in Medical Aids.shp has 63 Health_services 
in Pharmacies.shp has 122 Health_services 
there is  200 Health_services
run on Leisure_amenities
in Art Galeries.shp has 73 Leisure_amenities 
in Cinemas.shp has 6 Leisure_amenities 
in Museums.shp has 24 Leisure_amenities 
in Music Centers.shp has 13 Leisure_amenities 
in Theaters.shp has 13 Leisure_amenities 
there is  129 Leisure_amenities
run on Playgrounds
in Playgrounds.shp has 385 Playgrounds 
there is  385 Playgrounds
run on Sport_facilities
in Garden Sports Facilities.shp has 154 Sport_facilities 
in Gyms.shp has 39 Sport_facilities 
in Sport Lots.shp has 407 Sport_facilities 
in Sports Halls.shp has 102 Sport_facilities 
in Stadiums.shp has 6 Sport_facilities 
in Swimming Pools.shp has 17 Sport_facili

In [17]:
# The network to work with
res_path = 'output/streets_elements'
graph_path = f'{res_path}/elements/graph/'
clean_network = gpd.read_file(r'{}/streets_elements.shp'.format(res_path))
network_for_graph = gpd.read_file(f'{graph_path}/split_network.shp')
col_to_leave = ['oidrechov','length','geometry']

# The new network will contain more information for each object and will also be transformed into a graph
clean_network_temp =clean_network[col_to_leave]
G = momepy.gdf_to_nx(network_for_graph, approach="primal", length='length')
nodes, edges, W = momepy.nx_to_gdf(G, spatial_weights=True)

C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 29 disconnected components.
  warnings.warn(message)


In [15]:
# Assign graph elements to a SHP file
nodes.to_file('{}/elements/graph/nodes.shp'.format(res_path))
edges.to_file('{}/elements/graph/edges.shp'.format(res_path))

In [6]:
# For a given element, this function calculates the closet node between two closet edge nodes
def find_closet_pnt(row):
    start_pnt = nodes.loc[row['node_start']].geometry
    end_pnt = nodes.loc[row['node_end']].geometry
    if row.geometry.distance(start_pnt) <row.geometry.distance(end_pnt):
        return row['node_start']
    else:
         return row['node_end']

In [7]:
# "ego_graph" is used to calculate all edges whose paths to them from the given node (stored in row variable) are shorter than the radius value.
def find_close_edges(row, radius):
    flag =True
    node_id = row['nodeID']
    node_geometry = nodes[nodes['nodeID']==node_id]['geometry'].to_list()[0]
    G_temp = nx.generators.ego_graph(G, (node_geometry.x,node_geometry.y), radius=radius, distance='length')
    try:
        nodes_0, edges_0, W = momepy.nx_to_gdf(G_temp, spatial_weights=True)
    except ValueError:
        # In the case of a single edge
        temp_row_id = row['oidrechov']
        print('{}:only one polyline is found'.format(node_id))
        temp_count = clean_network_temp[clean_network_temp['oidrechov']==temp_row_id][count_field]
        clean_network_temp[clean_network_temp['oidrechov']==temp_row_id][count_field]= temp_count +1
        flag = False
    if flag:
        # Add one to all the founded edges (which denote that these streets are close to the given object)
        clean_network_temp.loc[clean_network_temp['oidrechov'].isin(edges_0['oidrechov']),count_field] = clean_network_temp.loc[clean_network_temp['oidrechov'].isin(edges_0['oidrechov']),count_field] +1

In [18]:
# The code should be applied to all objects in the "files_count_as_network" list.
files_count_as_network = ['educational_institutes','synagogues','Leisure_amenities','Sport_facilities','Health_services','Playgrounds']
for name in files_count_as_network:
    print(name)
    path = '{}/elements/elements_as_graph/{}'.format(res_path,name)
    path2 = '{}/detailed_folder/elements_as_graph/{}'.format(res_path,name)
    object_file = gpd.read_file('{}.shp'.format(path))
    # Find the closet node for each object
    sjoin= object_file.sjoin_nearest(edges,distance_col='dis')
    sjoin['nodeID'] = sjoin.apply(find_closet_pnt,axis=1)
    os.makedirs(path2,exist_ok = True)
    sjoin.drop_duplicates(subset=['index', 'nodeID'],inplace=True)
    sjoin.to_file('{}/sjoin.shp'.format(path2))

    radiuses = [400,500,600]
    for val in radiuses:
    # Use one of the radiuses to apply the closet edges in the graph
        print(val)
        count_field = 'count{}'.format(val)
        clean_network_temp[count_field] = 0
        sjoin.apply(lambda x:find_close_edges(x,val),axis=1)
    clean_network_temp.to_file('{}/{}.shp'.format(path2,name))
    clean_network[name] = clean_network_temp['count400']
clean_network.to_file(r'{}/streets_elements_as_graph.shp'.format(res_path))
print('Finish')

educational_institutes


C:\Users\Achituv\AppData\Local\Temp\ipykernel_13936\2495793.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sjoin.to_file('{}/sjoin.shp'.format(path2))
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


400
6540:only one polyline is found
6540:only one polyline is found
500


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


6540:only one polyline is found
6540:only one polyline is found
600
6540:only one polyline is found
6540:only one polyline is found
synagogues
400


...

500
600
Playgrounds
400


C:\Users\Achituv\AppData\Local\Temp\ipykernel_13936\2495793.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sjoin.to_file('{}/sjoin.shp'.format(path2))


500
600


C:\Users\Achituv\AppData\Local\Temp\ipykernel_13936\2495793.py:24: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clean_network.to_file(r'{}/streets_elements_as_graph.shp'.format(res_path))


Finish
